In [187]:
import numpy as np

good_patterns = [[1, 0, 0, 1]]
training_data_length = 1000

def getTrainingData(length, pattern_length):
    training_data = np.array([np.array([0, 1]).repeat(pattern_length / 2)] * length)
    
    for pattern in training_data:
        np.random.shuffle(pattern)   
    
    return training_data

def getCorrectResults(patterns):
    return np.array([1 if list(pattern) in good_patterns else 0 for pattern in patterns])

training_data = getTrainingData(training_data_length, pattern_length=4)
correct_results_training_data = getCorrectResults(training_data)

In [188]:
import tensorflow.keras as keras


NN = keras.Sequential()
NN.add(keras.Input(shape=(4,)))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=150)

NN.summary()

weights = NN.get_weights()

# print the weights
for i, layer_weights in enumerate(weights):
    print(f"Layer {i}:")
    for j, weight in enumerate(layer_weights):
        print(f"Weight {j}: {weight}")

Epoch 1/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step - accuracy: 0.3149 - loss: 0.8884 
Epoch 2/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - accuracy: 0.3229 - loss: 0.8641
Epoch 3/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - accuracy: 0.4577 - loss: 0.8151
Epoch 4/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6845 - loss: 0.7702
Epoch 5/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - accuracy: 0.6873 - loss: 0.7351
Epoch 6/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - accuracy: 0.6878 - loss: 0.7040
Epoch 7/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.6621 - loss: 0.6913
Epoch 8/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.6931 - loss: 0.6500
Epoch 9/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - accuracy: 0.6647 - loss: 0.6344
Epoch 10/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - accuracy: 0.6748 - loss: 0.6138
Epoch 11/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - accuracy: 0.6679 - loss: 0.5927
Epoch 12/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_73 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17 (72.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12 (52.00 B)

Layer 0:
Weight 0: [1.6572231]
Weight 1: [-3.08476]
Weight 2: [-2.229526]
Weight 3: [1.5552403]
Layer 1:
Weight 0: -1.7078230381011963


Now we want to test the Network with a  new list of patterns.

In [189]:
test_data = np.copy(training_data)
np.random.shuffle(test_data)

# NN.predict returns an array with shape=(trainging_data_length = 1000, output_neurons = 1) 
# so we flatten it to get a 1D array with our predictions
results_on_test_data = NN.predict(test_data).flatten() 
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print(f"all test data correct, when rounded: {all_correct}")
print(f"maximum difference from correct result: {max_difference}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
all test data correct, when rounded: True
maximum difference from correct result: 0.18173444271087646


In [211]:
good_patterns = [
    list(np.array(list(zip(np.ones(32), np.zeros(32)))).flatten()),
    list(np.array(list(zip(np.zeros(32), np.ones(32)))).flatten())
]

training_data_length = 50_000

def getTrainingData(length, pattern_length):
    training_data = np.array([np.array([0, 1]).repeat(pattern_length / 2)] * length)
    
    for i, pattern in enumerate(training_data):
        # this time we additionally inject some good patterns, 
        # because otherwise there wouldnt be enough good patterns in training_data
        rand = np.random.rand()
        if rand < 0.2:
            training_data[i] = np.array(good_patterns[0], dtype=int)
        elif rand < 0.4:
            training_data[i] = np.array(good_patterns[0], dtype=int)
        else: 
            np.random.shuffle(pattern) # shuffle in place, pattern is still the actual element from the training_data array
    
    return training_data

training_data = getTrainingData(training_data_length, pattern_length=64)
correct_results_training_data = getCorrectResults(training_data)

In [212]:
NN = keras.Sequential()
NN.add(keras.Input(shape=(64,)))
NN.add(keras.layers.Dense(32, activation="relu"))
NN.add(keras.layers.Dense(16, activation="relu"))
NN.add(keras.layers.Dense(8, activation="relu"))
NN.add(keras.layers.Dense(4, activation="relu"))
NN.add(keras.layers.Dense(2, activation="relu"))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=35)

NN.summary()

Epoch 1/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 767us/step - accuracy: 0.9394 - loss: 0.2548
Epoch 2/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 1.0000 - loss: 0.0784
Epoch 3/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 677us/step - accuracy: 1.0000 - loss: 0.0345
Epoch 4/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 718us/step - accuracy: 1.0000 - loss: 0.0163
Epoch 5/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - accuracy: 1.0000 - loss: 0.0083
Epoch 6/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 744us/step - accuracy: 1.0000 - loss: 0.0043
Epoch 7/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 789us/step - accuracy: 1.0000 - loss: 0.0022
Epoch 8/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 747us/step - accuracy: 1.0000 - loss: 0.0012
Epoch 9/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 1.0000 - loss: 6.0006e-04
Epoch 10/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 733us/step - accuracy: 1.0000 - loss: 3.2129e-04
Epoch 11/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 735us/step - accuracy: 1.0000 - loss: 1.6648e

Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_110 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_111 (Dense)               │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_112 (Dense)               │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_113 (Dense)               │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_114 (Dense)               │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_115 (Dense)               │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,381 (32.74 KB)

 Trainable params: 2,793 (10.91 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,588 (21.83 KB)

In [213]:
test_data = getTrainingData(500_000, pattern_length=64)

results_on_test_data = NN.predict(test_data).flatten()
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print(f"all test data correct, when rounded: {all_correct}")
print(f"maximum difference from correct result: {max_difference}")

15625/15625 ━━━━━━━━━━━━━━━━━━━━ 7s 457us/step
all test data correct, when rounded: True
maximum difference from correct result: 5.858555596205406e-06
